In [ ]:
# Model calling and intial setup
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser 
import warnings
warnings.filterwarnings("ignore") 

load_dotenv()
# Load env
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
AZURE_BASE_URL = os.getenv("AZURE_BASE_URL")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_CHAT_DEPLIOYMENT_NAME = os.getenv("AZURE_CHAT_DEPLIOYMENT_NAME")

parser = StrOutputParser()

llm_gemini = ChatGoogleGenerativeAI(model="gemini-2.0-flash" , api_key= GOOGLE_API_KEY)

llm_openai = AzureChatOpenAI(
    model="gpt-4o-mini",                         
    deployment_name=AZURE_CHAT_DEPLIOYMENT_NAME ,  # deployment name in Azure
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_BASE_URL,
    api_version="2024-02-01"
) 
llm_openai.invoke("What are your creater, also what type of LLM are you").content
# llm_gemini.invoke("who is father of india").content

c:\Users\singh\Let's Gooooo\Langgraph\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


'I was created by OpenAI, and I am a language model based on the GPT (Generative Pre-trained Transformer) architecture. Specifically, I am designed to understand and generate human-like text based on the input I receive. My training involved processing a diverse range of text from the internet, books, and other sources, allowing me to respond to a wide variety of queries and topics.'

### LLM Iterative Workflow

In [ ]:
from langgraph.graph import StateGraph
from typing import TypedDict , Literal

class SmState(TypedDict):
    topic : str
    post : str
    eval : Literal['approved' , "need_impro"]
    feedback : str
    iter_ : int
    max_iter : int = 3
    
def get_post(state:SmState)->SmState:
    prompt = PromptTemplate(
        template="You're a LinkedIN writter, Insprired from the modern posts writing, you have to write a 100-150 word ppst on the topic of {topic}. Make no mistake and factual Error",
        input_variables=['topic']
    )
    
    # Creating Chains
    post_chain = prompt | llm_openai | parser
    # Invoke chain
    post = post_chain.invoke({'topic': state['topic']})
    state['post'] = post
    return state

def evaluate_post(state:SmState)->SmState:
    
    return state

def optimise_post(state:SmState)->SmState:
    
    return state